# Machine Learning is not that complicated (in Python)

In [121]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn import cross_validation
from sklearn import metrics

data set: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [291]:
labels=[]
documents=[]
with open('./reviews.csv') as f:
    for line in f:
        line_list = line.split('|')
        if line_list[3] == '1' or line_list[3] == '2':
            labels.append('neg')
            documents.append(line_list[4])

        elif line_list[3] == '4' or line_list[3] == '5':
            labels.append('pos')
            documents.append(line_list[4])


len(documents)

8439

In [292]:
for document, label in zip(documents, labels)[:3]:
    print document
    print label
    print

This book was a bizarre experience for me. It reads much like a traditional, classic English novel, except with loads of descriptive sex and vulgar words mixed in for shock value. Instead of being shocked, though, I just found it all a bit tiresome and rather silly.  Maybe it was the fact that Lawrence sometimes used words like "thee" and "thy" and "dost" mixed in with modern day vulgarities that added to the overall unintentional humor of it for me, or perhaps it was that the vulgarities were simply used so darned often. In any event, I found myself laughing out loud often. I also found myself cringing. C and F words aside, did anyone tell Lawrence the word "bowels" is not particularly appealing? Anyway, I can see why some people felt at the time this was quite simply a trashy romance disguised as literature. It kind of is. Well-written and intelligent, for the most part, but still a bit trashy nonetheless. The story is essentially this: Lady Chatterley's young husband is paralyzed fr

In [293]:
print 'number neg:', len([item for item in labels if item == 'neg'])

number neg: 2193


In [294]:
print 'number pos:', len([item for item in labels if item == 'pos'])

number pos: 6246


# transform texts into vectors

let's use TF-IDF (term frequency, inverse document frequency):

- give more weight to words that occur a lot within a document
- give less weight to words that occur in many documents

In [295]:
vectorizer = TfidfVectorizer()

In [296]:
X = vectorizer.fit_transform(documents)
y = np.array(labels)

print X.shape, y.shape

(8439, 73370) (8439,)


In [297]:
y

array(['neg', 'pos', 'pos', ..., 'pos', 'pos', 'pos'],
      dtype='|S3')

# instantiate classifier

naive Bayes:

$$probability(spam | document) = probability(document | spam) \times probability(spam) / probability(document)$$

$$ \approx prob(word_1|spam) \times prob(word_2|spam) \times ... \times prob(word_n|spam) \times prob(spam)$$

"naive" = "wrong"

In [298]:
clf = BernoulliNB()

# cross validation

In [299]:
cv = cross_validation.StratifiedKFold(y,5)

In [300]:
precision=[]
recall=[]
for train, test in cv:
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
#     print len(y_train), len(y_test)
#     clf = BernoulliNB()
    clf.fit(X_train, y_train)
    y_hat = clf.predict(X_test)
    p,r,_,_ = metrics.precision_recall_fscore_support(y_test, y_hat)
    precision.append(p[1])
    recall.append(r[1])

In [301]:
precision

[0.77249022164276404,
 0.76329442282749671,
 0.75631885936487364,
 0.7480053191489362,
 0.76290760869565222]

# average precision / recall across k-folds

- precision: of predicted negative reviews, how many are actually negative reviews?
- recall: of the actual negative reviews, how many are predicted to be negative reviews?

In [302]:
print vectorizer
print clf
print 'precision:',np.average(precision), '+/-', np.std(precision)
print 'recall:', np.average(recall), '+/-', np.std(recall)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
precision: 0.760603286336 +/- 0.00813531117853
recall: 0.924908246597 +/- 0.0208652428031


# try on new negative review

In [303]:
neg_sample = 'yuck sickening overblown yawn overuse scrooge unoriginal'
neg_sample = vectorizer.transform([neg_sample])
#print sample

In [304]:
clf.predict_proba(neg_sample)

array([[  9.99999919e-01,   8.10008685e-08]])

In [305]:
pos_sample = 'delightful masterfully accessible bittersweet examines terrific playful'
pos_sample = vectorizer.transform([pos_sample])

In [306]:
clf.predict_proba(pos_sample)

array([[  1.18745218e-07,   9.99999881e-01]])

# most spammy words

In [307]:
stop_words = vectorizer.get_stop_words()
len(stop_words)

TypeError: object of type 'NoneType' has no len()

In [308]:
probs=clf.feature_log_prob_[1] - clf.feature_log_prob_[0]
len(probs)

73370

In [309]:
features=vectorizer.get_feature_names()
len(features)

73370

In [310]:
sorted(zip(probs,features))[:70]


[(-3.5305367330144737, u'dnf'),
 (-3.2428546605626938, u'overuse'),
 (-2.9915402322817872, u'coworkers'),
 (-2.9915402322817872, u'sylvia'),
 (-2.9915402322817872, u'unoriginal'),
 (-2.8373895524545283, u'congratulatory'),
 (-2.8373895524545283, u'faire'),
 (-2.8373895524545283, u'grossed'),
 (-2.8373895524545283, u'letdown'),
 (-2.8373895524545283, u'rebus'),
 (-2.6550679956605743, u'abortions'),
 (-2.6550679956605743, u'amateurish'),
 (-2.6550679956605743, u'anagrams'),
 (-2.6550679956605743, u'astronaut'),
 (-2.6550679956605743, u'beide'),
 (-2.6550679956605743, u'catholics'),
 (-2.6550679956605743, u'concur'),
 (-2.6550679956605743, u'couldnt'),
 (-2.6550679956605743, u'daft'),
 (-2.6550679956605743, u'irgendwie'),
 (-2.6550679956605743, u'mistero'),
 (-2.6550679956605743, u'praiseworthy'),
 (-2.6550679956605743, u'selflessly'),
 (-2.6550679956605743, u'slogged'),
 (-2.6550679956605743, u'storica'),
 (-2.6550679956605743, u'wannabe'),
 (-2.6550679956605743, u'yadda'),
 (-2.54970748

![](cat.jpg)

In [311]:
sorted(zip(probs,features), reverse=True)[:70]


[(2.5919560764999119, u'madame'),
 (2.5919560764999119, u'delightful'),
 (2.5919560764999119, u'bovary'),
 (2.5378888552296361, u'heroines'),
 (2.3883571212586725, u'nazis'),
 (2.3883571212586725, u'congo'),
 (2.3883571212586725, u'coast'),
 (2.3720966003868922, u'extraordinary'),
 (2.3555672984356821, u'vonnegut'),
 (2.3555672984356821, u'terrific'),
 (2.3555672984356821, u'masterfully'),
 (2.3555672984356821, u'ease'),
 (2.3216657467600008, u'infamous'),
 (2.3216657467600008, u'amazingly'),
 (2.2865744269487305, u'samuel'),
 (2.2865744269487305, u'bittersweet'),
 (2.2502067827778554, u'varied'),
 (2.2502067827778554, u'playful'),
 (2.2502067827778554, u'lean'),
 (2.2502067827778554, u'gem'),
 (2.2502067827778554, u'condemned'),
 (2.2124664547950079, u'tim'),
 (2.2124664547950079, u'occupation'),
 (2.2124664547950079, u'housekeeper'),
 (2.2124664547950079, u'hardships'),
 (2.2124664547950079, u'chaucer'),
 (2.193048368937907, u'visited'),
 (2.1732457416417272, u'september'),
 (2.17324

AttributeError: 'list' object has no attribute 'to_csv'